In [140]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import calendar

In [67]:
test_date = dt.datetime(2017,8,25)

In [80]:
df = pd.read_csv("transactions.csv")
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index(df['Date']).drop('Date',axis=1)
df.head()

,Description,Original Description,Amount,Transaction Type,Category,Account Name,Labels,Notes
Date,,,,,,,,
2017-09-09,Gig Car Share,GIG CAR SHARE,0.50,debit,Rental Car & Taxi,Discover,NaN,NaN
2017-09-09,Gig Car Share,GIG CAR SHARE,0.50,debit,Rental Car & Taxi,Discover,NaN,NaN
2017-09-10,Lyft,LYFT *RIDE SUN 2PM 855-280-0278 CA,15.38,debit,Rental Car & Taxi,Discover,NaN,NaN
2017-09-10,Lyft,LYFT *RIDE SUN 4PM 855-280-0278 CA,10.88,debit,Rental Car & Taxi,Discover,NaN,NaN
2017-09-09,Cafe Durant,CAFE DURANT BERKELEY CA,9.66,debit,Restaurants,Discover,NaN,NaN


In [51]:
to = dt.datetime.today()

In [139]:
categories = set(df['Category'])
print(categories)

{'Pharmacy', 'Public Transportation', 'Tuition', 'Gift', 'Advertising', 'Sporting Goods', 'Restaurants', 'Clothing', 'Transfer for Cash Spending', 'Rental Car & Taxi', 'Television', 'Groceries', 'Credit Card Payment', 'Air Travel', 'Doctor', 'Gym', 'Fast Food', 'Alcohol & Bars', 'Food & Dining', 'Shopping', 'Mortgage & Rent', 'Hair', 'Coffee Shops'}


In [72]:
def getTodayExpenditure(tab, today=dt.datetime.today()):
    '''gets amount of money spent toady'''
    today_transactions = tab.loc[today.date()]
    return sum(today_transactions['Amount'])

In [79]:
test = getTodayExpenditure(df,test_date)
print("$",round(test,2))

$ 85.64


In [165]:
def getRangeExpenditure(tab, date_start, date_end, category=None):
    '''returns the amount of money spent in date range'''
    today_transactions = tab.loc[[i for i in pd.date_range(date_start, date_end).values if i in tab.index.values]]
    today_transactions.head()
    if (category==None):
        return sum(today_transactions['Amount'])
    else:
        return sum(today_transactions[today_transactions['Category']==category]['Amount'])

In [166]:
getRangeExpenditure(df, (dt.datetime.today()-dt.timedelta(days=60)).date(),dt.datetime.today().date(), 'Restaurants')

129.41000000000003

In [225]:
def getFrequencies(tab, start_day, end_day, freq='daily'):
    transaction_range = tab.loc[[i for i in pd.date_range(start_day, end_day).values if i in tab.index.values]]
    out = df.groupby('Category')['Amount'].sum().to_dict()
    if (freq == 'weekly'):
        r = (end_day - start_day).days/7
    elif (freq=='monthly'):
        r = (end_day - start_day).days/30
    elif (freq=='yearly'):
        r = (end_day - start_day).days/365
    else:
        r = (end_day - start_day).days
    for k in out:
        out[k] = out[k]/r
    return out

In [226]:
def getFrequenciesCounts(tab, start_day, end_day, freq='daily'):
    transaction_range = tab.loc[[i for i in pd.date_range(start_day, end_day).values if i in tab.index.values]]
    out = transaction_range['Category'].value_counts().to_dict()
    if (freq == 'weekly'):
        r = (end_day - start_day).days/7
    elif (freq=='monthly'):
        r = (end_day - start_day).days/30
    elif (freq=='yearly'):
        r = (end_day - start_day).days/365
    else:
        r = (end_day - start_day).days
    for k in out:
        out[k] = out[k]/r
    return out

In [227]:
getFrequencies(df, test_date, to)

{'Advertising': 3.5714285714285716,
 'Air Travel': 4.4380952380952383,
 'Alcohol & Bars': 0.51761904761904753,
 'Clothing': 2.6066666666666669,
 'Coffee Shops': 2.9023809523809527,
 'Credit Card Payment': 237.95380952380953,
 'Doctor': 0.95238095238095233,
 'Fast Food': 13.238095238095244,
 'Food & Dining': 3.6738095238095241,
 'Gift': 1.3552380952380954,
 'Groceries': 35.310000000000002,
 'Gym': 3.3333333333333335,
 'Hair': 0.95238095238095233,
 'Mortgage & Rent': 2.3809523809523809,
 'Pharmacy': 1.5080952380952382,
 'Public Transportation': 7.75,
 'Rental Car & Taxi': 12.722380952380956,
 'Restaurants': 40.12714285714285,
 'Shopping': 80.959523809523787,
 'Sporting Goods': 4.6404761904761909,
 'Television': 8.3309523809523807,
 'Transfer for Cash Spending': 7.6190476190476186,
 'Tuition': 3.3333333333333335}

In [201]:
def getOccurrencies(tab, start_day, end_day, freq='daily'):
    transaction_range = tab.loc[[i for i in pd.date_range(start_day, end_day).values if i in tab.index.values]]
    out = transaction_range['Category'].value_counts().to_dict()
    return out

In [154]:
def getFrequencyDate(time, freq='daily'):
    """returns the start and end dates based on the given frequency
    possible freq = 'daily', 'weekly', 'monthly', 'yearly' """
    if (freq=='weekly'):
        start = time - dt.timedelta(days=time.weekday())
        end = start + dt.timedelta(days=6)
    elif (freq=='monthly'):
        start = time.replace(day=1)
        start_day, end_day = calendar.monthrange(time.year,time.month)
        end = time.replace(day=end_day)
    elif (freq=='yearly'):
        start = time.replace(month=1).replace(day=1)
        end = time.replace(month=12).replace(day=31)
    else:
        start = time
        end = time
    return start, end

In [232]:
overall_f = getFrequencies(df, min(df.index), max(df.index), 'monthly')

In [233]:
overall_f

{'Advertising': 6.4102564102564106,
 'Air Travel': 7.9658119658119668,
 'Alcohol & Bars': 0.92905982905982909,
 'Clothing': 4.6786324786324789,
 'Coffee Shops': 5.2094017094017095,
 'Credit Card Payment': 427.09658119658121,
 'Doctor': 1.7094017094017095,
 'Fast Food': 23.760683760683772,
 'Food & Dining': 6.5940170940170946,
 'Gift': 2.4324786324786327,
 'Groceries': 63.376923076923077,
 'Gym': 5.982905982905983,
 'Hair': 1.7094017094017095,
 'Mortgage & Rent': 4.2735042735042734,
 'Pharmacy': 2.706837606837607,
 'Public Transportation': 13.910256410256411,
 'Rental Car & Taxi': 22.835042735042741,
 'Restaurants': 72.023076923076914,
 'Shopping': 145.31196581196579,
 'Sporting Goods': 8.3290598290598297,
 'Television': 14.952991452991453,
 'Transfer for Cash Spending': 13.675213675213676,
 'Tuition': 5.982905982905983}